In [1]:
import rioxarray
import numpy as np
import os
from glob import glob
from tqdm.notebook import tqdm

In [2]:
images = sorted(glob("../data/Sample1_tile/image/*.tif"))
masks = sorted(glob("../data/Sample1_tile/mask/*.tif"))

In [3]:
X = []
Y = []
samples = 100
count = 0
for image, mask in tqdm(zip(images, masks)):
    _,image_name = os.path.split(image)
    _,mask_name = os.path.split(mask)
    image_name,_ = os.path.splitext(image_name)
    mask_name,_ = os.path.splitext(mask_name)
    image_name = image_name.split('_')
    mask_name = mask_name.split('_')
    if(image_name[1] != mask_name[1] and image_name[2] != mask_name[2]):
        raise NameError(f"{image_name=} {mask_name=}")
    
    x = rioxarray.open_rasterio(image)
    y = rioxarray.open_rasterio(mask)
    X.append(np.expand_dims(x.data, axis=0))
    Y.append(y.data)
    count += 1
    if(count >= samples):
        break


0it [00:00, ?it/s]

In [4]:
X = np.vstack(X).astype(np.float32)
Y = np.vstack(Y)
X.shape, Y.shape

((100, 3, 512, 512), (100, 512, 512))

In [5]:
os.environ["SM_FRAMEWORK"] = "tf.keras"
import segmentation_models as sm

2024-07-22 08:35:17.298879: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-22 08:35:17.310370: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-22 08:35:17.313881: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-22 08:35:17.322761: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-22 08:35:19.587903: W tensorflow/compiler/tf2

Segmentation Models: using `tf.keras` framework.


In [7]:
from segmentation_models import Unet
from segmentation_models import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score
import keras

BACKBONE = 'inceptionresnetv2'
preprocess_input = get_preprocessing(BACKBONE)

# load your data
# x_train, y_train, x_val, y_val = load_data(...)

# preprocess input
X = preprocess_input(X)
# define model
model = Unet(BACKBONE, classes=1, encoder_weights='imagenet', encoder_freeze=True,)
loss = sm.losses.DiceLoss()
metrics = [sm.metrics.IOUScore(),'binary_accuracy',keras.metrics.Precision(),keras.metrics.Recall()]
model.compile("Adam", loss=loss, metrics=metrics)

# fit model
model.fit(
    x=X,
    y=Y,
    batch_size=1,
    epochs=100,
    validation_data=(X, Y),
)

Epoch 1/100


ValueError: Input 0 of layer "functional_1" is incompatible with the layer: expected shape=(None, None, None, 3), found shape=(1, 3, 512, 512)

In [70]:
model.weights

[<KerasVariable shape=(3, 3, 3, 32), dtype=float32, path=conv2d_1015/kernel>,
 <KerasVariable shape=(32,), dtype=float32, path=batch_normalization_1015/beta>,
 <KerasVariable shape=(32,), dtype=float32, path=batch_normalization_1015/moving_mean>,
 <KerasVariable shape=(32,), dtype=float32, path=batch_normalization_1015/moving_variance>,
 <KerasVariable shape=(3, 3, 32, 32), dtype=float32, path=conv2d_1016/kernel>,
 <KerasVariable shape=(32,), dtype=float32, path=batch_normalization_1016/beta>,
 <KerasVariable shape=(32,), dtype=float32, path=batch_normalization_1016/moving_mean>,
 <KerasVariable shape=(32,), dtype=float32, path=batch_normalization_1016/moving_variance>,
 <KerasVariable shape=(3, 3, 32, 64), dtype=float32, path=conv2d_1017/kernel>,
 <KerasVariable shape=(64,), dtype=float32, path=batch_normalization_1017/beta>,
 <KerasVariable shape=(64,), dtype=float32, path=batch_normalization_1017/moving_mean>,
 <KerasVariable shape=(64,), dtype=float32, path=batch_normalization_1017